In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv(r"C:\Users\hp\Desktop\titanic_kaggel\train.csv")

In [3]:
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [10]:
columns_to_drop=['PassengerId','Name','Ticket','Cabin','Embarked']

In [12]:
data_cleaned=data.drop(columns_to_drop,axis=1)

In [13]:
data_cleaned.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [25]:
x=data_cleaned.iloc[:,1:]
y=data_cleaned.iloc[:,0]

In [17]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data_cleaned['Sex']=le.fit_transform(data_cleaned['Sex'])

In [22]:
data_cleaned=data_cleaned.fillna(data_cleaned['Age'].mean())

In [23]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int32  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int32(1), int64(4)
memory usage: 45.4 KB


In [30]:
# model=RandomForestClassifier()
# model.fit(x,y)

In [31]:
# model.feature_importances_

In [32]:
print(x.shape,y.shape)

(891, 6) (891,)


In [33]:
def entropy(col):
    counts=np.unique(col,return_counts=True)
    n=col.shape[0]
    ent=0.0
    
    for xi in counts[1]:
        p=xi/n
        ent+=-1.0*p*np.log2(p)
        
    return ent

In [42]:
def divide_data(x_data,fkey,fval):
    x_ryt=pd.DataFrame([],columns=x_data.columns)
    x_lft=pd.DataFrame([],columns=x_data.columns)
    
    for ix in range(x_data.shape[0]):
        val=x_data[fkey].loc[ix]
        
        if val>fval:
            x_ryt=x_ryt.append(x_data.loc[ix])
        else:
            x_lft=x_lft.append(x_data.loc[ix])
            
    return x_lft,x_ryt

In [47]:
def information_gain(x_data,fkey,fval):
    left,right=divide_data(x_data,fkey,fval)
    
    l=float(left.shape[0])/x_data.shape[0]
    r=float(right.shape[0])/x_data.shape[0]
    
    #if all examples come to one side
    if left.shape[0]==0 or right.shape[0]==0:
        return -100000 #min IG
    
    i_gain=entropy(x_data.Survived)-(l*entropy(left.Survived)+r*entropy(right.Survived))
    
    return i_gain

In [49]:
for fx in x.columns:
    print(fx)
    print(information_gain(data_cleaned,fx,data_cleaned[fx].mean()))

Pclass
0.07579362743608165
Sex
0.2176601066606142
Age
0.001158644038169343
SibSp
0.009584541813400071
Parch
0.015380754493137694
Fare
0.042140692838995464


In [50]:
class DecisionTree:
    
    #Constructor
    def __init__(self,depth=0,max_depth=5):
        self.left = None
        self.right = None
        self.fkey = None
        self.fval = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
        
    def train(self,X_train):
        
        features = ['Pclass','Sex','Age','SibSp', 'Parch', 'Fare']
        info_gains = []
        
        for ix in features:
            i_gain = information_gain(X_train,ix,X_train[ix].mean())
            info_gains.append(i_gain)
            
        self.fkey = features[np.argmax(info_gains)]
        self.fval = X_train[self.fkey].mean()
        print("Making Tree Features is",self.fkey)
        
        #Split Data
        data_left,data_right = divide_data(X_train,self.fkey,self.fval)
        data_left = data_left.reset_index(drop=True)
        data_right = data_right.reset_index(drop=True)
         
        #Truly a left node
        if data_left.shape[0]  == 0 or data_right.shape[0] ==0:
            if X_train.Survived.mean() >= 0.5:
                self.target = "Survive"
            else:
                self.target = "Dead"
            return
        #Stop earyly when depth >=max depth
        if(self.depth>=self.max_depth):
            if X_train.Survived.mean() >= 0.5:
                self.target = "Survive"
            else:
                self.target = "Dead"
            return
        
        #Recursive Case
        self.left = DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(data_left)
        
        self.right = DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(data_right)
        
        #You can set the target at every node
        if X_train.Survived.mean() >= 0.5:
            self.target = "Survive"
        else:
            self.target = "Dead"
        return
    
    def predict(self,test):
        if test[self.fkey]>self.fval:
            #go to right
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)

In [52]:
split = int(0.7*data_cleaned.shape[0])
train_data = data_cleaned[:split]
test_data = data_cleaned[split:]
test_data = test_data.reset_index(drop=True)

In [53]:
print(train_data.shape,test_data.shape)

(623, 7) (268, 7)


In [54]:
dt = DecisionTree()

In [55]:
dt.train(train_data)

Making Tree Features is Sex
Making Tree Features is Pclass
Making Tree Features is Age
Making Tree Features is SibSp
Making Tree Features is Pclass
Making Tree Features is Age
Making Tree Features is Age
Making Tree Features is SibSp
Making Tree Features is Parch
Making Tree Features is Pclass
Making Tree Features is SibSp
Making Tree Features is Fare
Making Tree Features is Parch
Making Tree Features is Age
Making Tree Features is Pclass
Making Tree Features is Age
Making Tree Features is Age
Making Tree Features is Parch
Making Tree Features is SibSp
Making Tree Features is Fare
Making Tree Features is Age
Making Tree Features is Age
Making Tree Features is Fare
Making Tree Features is Age
Making Tree Features is Age
Making Tree Features is Fare
Making Tree Features is Age
Making Tree Features is Parch
Making Tree Features is Fare
Making Tree Features is Fare
Making Tree Features is Fare
Making Tree Features is Age
Making Tree Features is Fare
Making Tree Features is Parch
Making Tre

In [56]:
print(dt.fkey)
print(dt.fval)
print(dt.left.fkey)
print(dt.right.fkey)

Sex
0.6292134831460674
Pclass
Fare


In [74]:
y_pred = []
for ix in range(test_data.shape[0]):
    y_pred.append(dt.predict(test_data.loc[ix]))

In [75]:
y_pred

['Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'De

In [76]:
y_actual = test_data["Survived"]

In [77]:
le = LabelEncoder()
y_pred = le.fit_transform(y_pred)

In [78]:
y_pred

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0], dtype=int64)

In [81]:
acc = np.sum(np.array(y_pred)==np.array(y_actual))/y_pred.shape[0]

In [82]:
acc

0.8171641791044776

## Decision Tree using Sklearn

In [83]:
from sklearn.tree import DecisionTreeClassifier

In [106]:
sk_tree = DecisionTreeClassifier(criterion='entropy',max_depth=5)

In [107]:
input_cols=['Pclass',"Sex","Age","SibSp","Parch","Fare"]
output_cols=["Survived"]
sk_tree.fit(train_data[input_cols],train_data[output_cols])

DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [87]:
sk_tree.predict(test_data[input_cols])

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0], dtype=int64)

In [88]:
sk_tree.score(test_data[input_cols],test_data[output_cols])

0.8171641791044776

In [89]:
DecisionTreeClassifier?


## Visualise a Decison Tree

In [93]:
import pydotplus

# from sklearn.externals.six import StringIO
from six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz

In [94]:
# !pip install pydotplus

In [99]:
dot_data = StringIO()
export_graphviz(sk_tree,out_file=dot_data,filled=True,rounded=True)

In [105]:
# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
# Image(graph.create_png())

In [103]:
# !conda install python-graphviz

## one more implementation of decision tree but not for continuous featuree

In [123]:
#import basic libraries
import pandas as pd
import numpy as np
from numpy import log2 as log
eps = np.finfo(float).eps

#dataset 'S' given in the question
feature_data = pd.DataFrame({"Toothed":[1,1,0,1,1,1,1,1,0,1],"Hair":[1,0,1,1,1,0,0,1,0,1],"Breathes":[1,1,1,1,1,0,1,1,1,1],
                             "Legs":[1,0,1,1,1,0,0,1,1,1],
                             "Species":["Mammal","Reptile","Mammal","Mammal","Mammal","Reptile","Reptile","Mammal","Reptile","Mammal"]}, 
                              columns=["Toothed","Hair","Breathes","Legs","Species"])
features = feature_data[["Toothed","Hair","Breathes","Legs"]]
target = feature_data["Species"]

In [124]:
#calculate entropy of label
def calc_entropy(df):
    class_val = df.keys()[-1] 
    values = df[class_val].unique()
    entropy = 0
    for value in values:
        f = df[class_val].value_counts()[value]/len(df[class_val])
        entropy += -f*np.log2(f)
    return entropy

#entropy of all the attributes
def entropy_att(df,attribute):
    class_val = df.keys()[-1]   
    target_values = df[class_val].unique()  
    values = df[attribute].unique()
    entropy = 0
    for value in values:
        entropy1 = 0
        for target_value in target_values:
                n = len(df[attribute][df[attribute] == value][df[class_val] == target_value])
                d = len(df[attribute][df[attribute] == value])
                f = n/(d+eps)
                entropy += -f*log(f+eps)
        fraction2 = d/len(df)
        entropy += -fraction2*entropy1
    return abs(entropy)

In [125]:
#function to get subtable of met conditions
#node is column name
#value is unique value of column

def get_subtable(df, node, value):
    return df[df[node] == value].reset_index(drop=True)

In [126]:
#feature with highest info gain

def max_gain(df):
    info = []
    for key in df.keys()[:-1]:
        info.append(calc_entropy(df)-entropy_att(df,key))
    return df.keys()[:-1][np.argmax(info)]

In [127]:
#function to make decision tree
def build_tree(df,tree=None): 

    #attribute with highest info gain
    att = max_gain(df)
    
    #get distinct particular value of the attribute 
    attvalue = np.unique(df[att])
    
    #empty dictionary for creating tree    
    if tree is None:                    
        tree={}
        tree[att] = {}
  
    #recrusive calling to make tree

    for value in attvalue:
        
        subtable = get_subtable(df,att,value)
        colvalue,counts = np.unique(subtable['Species'],return_counts=True)                        
        
        if len(counts)==1:
            #subset pruity
            tree[att][value] = colvalue[0]     

        else:        
            #recursive calling
            tree[att][value] = buildTree(subtable) 
                   
    return tree

tree = build_tree(feature_data)

In [128]:
tree

{'Hair': {0: 'Reptile', 1: 'Mammal'}}

In [129]:
#predict input variable
def prediction(inst,tree):

    for att in tree.keys():        
        
        value = inst[att]
        tree = tree[att][value]
        prediction = 0
            
        if type(tree) is dict:
            #value of prediction in prediction (recursive)
            prediction = prediction(inst, tree)
        
        else:
            #go back to tree
            prediction = tree 
            break;                            
        
    return prediction

In [130]:
df = {"Toothed":0,"Hair":0,"Breathes":1,"Legs":1}
instance = pd.Series(df)
prediction(instance,tree)

'Reptile'